# **Below is code to add synapse labelings for a list of [x,y,z] in csv format onto the original SWC file**

In [94]:
#Code to add synapse labelings for a list(CSV file) of [x,y,z] onto the original SWC file

import pandas as pd
import pdb
import numpy as numpy
from io import StringIO
from scipy.spatial import distance
Exc_fn = "//172.16.0.50/sonic/fred/pvBb_bla1/rois/n2_excitatorySyn.csv"
Inh_fn = "//172.16.0.50/sonic/fred/pvBb_bla1/rois/n2_inhibitorySyn.csv"
swc_fn = '//172.16.0.50/sonic/fred/pvBb_bla1/swc/n2.swc'
out_fn = "//172.16.0.50/sonic/fred/pvBb_bla1/swc_synAdded/n2_synAdded.swc"

exc_value = 1
inh_value = 2

swcData = numpy.genfromtxt(swc_fn,comments = "#", delimiter = ' ')

with open(Exc_fn, 'r', encoding='utf-8-sig') as f: 
    exc_synData = numpy.genfromtxt(f, dtype=float, delimiter=',')

with open(Inh_fn, 'r', encoding='utf-8-sig') as f: 
    inh_synData = numpy.genfromtxt(f, dtype=float, delimiter=',')
    
#Code to assign synapses to swc for excitatory
dist = distance.cdist(swcData[:,2:5],exc_synData[:],'euclidean')

for i in range(0,len(exc_synData)):
    j = 0;
    idx = 0;
    while True:
        idx = numpy.argpartition(dist[:,i],j)[j]
        if swcData[idx,7] == 0:
            swcData[idx,7] = exc_value;
            break
        j += 1
        
#Code to assign synapses to swc for inhibitory

dist = distance.cdist(swcData[:,2:5],inh_synData[:],'euclidean')

for i in range(0,len(inh_synData)):
    j = 0;
    idx = 0;
    while True:
        idx = numpy.argpartition(dist[:,i],j)[j]
        if swcData[idx,7] == 0:
            swcData[idx,7] = inh_value;
            break
        j += 1    

numpy.savetxt(out_fn,swcData, fmt = '%d', delimiter = ' ')


# **Below is code to convert SWC to ROI TSV format**

In [98]:
#Code to convert SWC to ROI TSV format
import numpy as numpy
swc_fn = "//172.16.0.50/sonic/fred/pvBb_bla1/swc_synAdded/n1_synAdded.swc"
roiTSVexc_fn = "//172.16.0.50/sonic/fred/pvBb_bla1/swc_synAdded/n1_excitatorySyn.tsv"
roiTSVinh_fn = "//172.16.0.50/sonic/fred/pvBb_bla1/swc_synAdded/n1_inhibitorySyn.tsv"
r = 5 #radius of circle

swcData = numpy.genfromtxt(swc_fn,comments = "#", delimiter = ' ')

idxExcitatory = numpy.where(swcData[:,7] == 1)
idxInhibitory = numpy.where(swcData[:,7] == 2)

roiExData = numpy.full((len(idxExcitatory[0]),4),r)
roiExData[:,0] = swcData[idxExcitatory,3]
roiExData[:,1] = swcData[idxExcitatory,2]
roiExData[:,2] = swcData[idxExcitatory,4]
roiInhData = numpy.full((len(idxInhibitory[0]),4),r)
roiInhData[:,0] = swcData[idxInhibitory,3]
roiInhData[:,1] = swcData[idxInhibitory,2]
roiInhData[:,2] = swcData[idxInhibitory,4]

numpy.savetxt(roiTSVexc_fn,roiExData, fmt = '%d', delimiter = '\t') 
numpy.savetxt(roiTSVinh_fn,roiInhData, fmt = '%d', delimiter = '\t') 


# Code to integrate distances along synapse

In [53]:
#Code to take nTracer SWC and convert to more general format (take centroid of soma)
#converts neurite type to dendrite
#now can handle batch folders

import pandas as pd
import pdb
import numpy as numpy
import os
from io import StringIO
from scipy.spatial import distance
from scipy import ndimage
swc_fn = "//172.16.0.50/sonic/fred/pvsom_bla5/SWC/som/"
out_fn = '//172.16.0.50/sonic/fred/pvsom_bla5/SWC/som/fixed/'
scaleXY = 0.15; #in microns
scaleZ = 0.35;


def processSWC(inFN,outFN):
    swcData = numpy.genfromtxt(inFN,comments = "#", delimiter = ' ')
    swcData[:,2] = swcData[:,2]*scaleXY;
    swcData[:,3] = swcData[:,3]*scaleXY;
    swcData[:,4] = swcData[:,4]*scaleZ;
    idxSoma = numpy.where(swcData[:,6] == -1)
    idxRest = numpy.where(swcData[:,6] >-1)
    #swcData[idxRest,6] = swcData[idxRest,6] - len(idxRest[0])

    somaCenter = numpy.mean(swcData[idxSoma,2:5],axis=1)
    newSwcData = numpy.zeros((len(idxRest[0])+1,7))

    newSwcData[0,:] = [1,1,0,0,0,20,-1]
    newSwcData[0,2:5] = somaCenter

    for i in range(0,len(idxRest[0])):
        newOrigin = swcData[idxRest[0][i],6] - len(idxSoma[0]) + 1
        if newOrigin < 1:
            newOrigin = 1
        newSwcData[i+1,:] = [i+2, 3,0,0,0,0,newOrigin]
        newSwcData[i+1,2:6] = swcData[idxRest[0][i],2:6]

    numpy.savetxt(outFN, newSwcData, fmt = '%d', delimiter = ' ')


if os.path.splitext(swc_fn)[1] != ".swc":
    list_swc = os.listdir(swc_fn)
    for filename in list_swc:
        if filename.endswith(".swc"):
            outPath = out_fn+'fixed_'+filename;
            print(filename);
            processSWC(swc_fn+filename,outPath);
else:
    processSWC(swc_fn,out_fn)

n1.swc
n15.swc
n19.swc
n2.swc
n22.swc
n23.swc
n49.swc


In [7]:
#Code to take nTracer SWC and scale by XY and Z factor for sholl analysis
#converts neurite type to dendrite
#now can handle batch folders

import pandas as pd
import pdb
import numpy as numpy
import os
from io import StringIO
from scipy.spatial import distance
from scipy import ndimage
swc_fn = "//172.16.0.50/sonic/fred/pvsom_bla5/SWC/pvcalb/"
out_fn = '//172.16.0.50/sonic/fred/pvsom_bla5/SWC/pvcalb/sholl/'
scaleXY = 0.15; #in microns
scaleZ = 0.35;


def processSWC(inFN,outFN):
    swcData = numpy.genfromtxt(inFN,comments = "#", delimiter = ' ')
    swcData[:,2] = swcData[:,2]*scaleXY;
    swcData[:,3] = swcData[:,3]*scaleXY;
    swcData[:,4] = swcData[:,4]*scaleZ;
    idxSoma = numpy.where(swcData[:,6] == -1)
    idxRest = numpy.where(swcData[:,6] >-1)
    #swcData[idxRest,6] = swcData[idxRest,6] - len(idxRest[0])
    swcData[idxRest,1] = 3
    numpy.savetxt(outFN, swcData, fmt = '%d', delimiter = ' ')


if os.path.splitext(swc_fn)[1] != ".swc":
    list_swc = os.listdir(swc_fn)
    for filename in list_swc:
        if filename.endswith(".swc"):
            outPath = out_fn+'sholl_'+filename;
            print(filename);
            processSWC(swc_fn+filename,outPath);
else:
    processSWC(swc_fn,out_fn)

n10.swc
n11.swc
n16.swc
n20.swc
n21.swc
n26.swc
n28.swc
n3.swc
n31.swc
n33.swc
n35.swc
n39.swc
n40.swc
n5.swc
n60.swc
n61.swc
n62.swc
n65.swc
n68.swc
n8.swc
n9.swc
